#### numpy教程：数学函数和基本统计函数：https://blog.csdn.net/pipisorry/article/details/41214819

#### pyecharts中的Overlap叠加不同类型图表输出:https://blog.csdn.net/qq_42379006/article/details/80841488

## 学生消费画像

In [1]:
# 忽略警告信息
import warnings
warnings.filterwarnings("ignore")
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
student_info=pd.read_csv(r"D:\Entertainment\Heaven Pool\education_data\2_student_info.csv")
consumption=pd.read_csv(r"D:\Entertainment\Heaven Pool\education_data\7_consumption.csv")

In [2]:
# 当前在校学生信息
student_info.columns=[u'学生id',u'学生姓名',u'性别',u'民族',u'出生日期',u'班级名',u'家庭住址',u'家庭类型',u'政治面貌',u'班级id',u'班级学期',u'是否住校',u'是否退学',u'宿舍号']
# student_info['是否退学'].fillna(value=0) #将NaN替换为0
#print(student_info.dtypes) # 查看dataframe中的数据类型
n_student=student_info['学生id'].unique().shape[0]
n_class=student_info['班级名'].unique().shape[0]
n_term=student_info['班级学期'].unique().shape[0]
print("学生信息_学生数量：",n_student)
print('学生信息_班级数量：',n_class)
print('学生信息_学期数量：',n_term)
print('学生信息_学期分布：',student_info['班级学期'].unique())
print("学生信息_是否退学：",student_info['是否退学'].unique())
student_info.head(5)

学生信息_学生数量： 1765
学生信息_班级数量： 46
学生信息_学期数量： 1
学生信息_学期分布： ['2018-2019-1']
学生信息_是否退学： [nan '1' ' ']


,学生id,学生姓名,性别,民族,出生日期,班级名,家庭住址,家庭类型,政治面貌,班级id,班级学期,是否住校,是否退学,宿舍号
0,14454,陈某某,女,汉族,2001.0,白-高二(01),宁波,城镇,共青团员,901,2018-2019-1,NaN,NaN,NaN
1,14479,曹某某,男,汉族,NaN,白-高二(01),NaN,城镇,共青团员,901,2018-2019-1,1.0,NaN,315.0
2,14486,金某某,男,汉族,NaN,白-高二(01),NaN,城镇,共青团员,901,2018-2019-1,1.0,NaN,321.0
3,14549,张某某,女,汉族,2002.0,白-高二(07),浙江平湖,城镇,共青团员,907,2018-2019-1,NaN,NaN,NaN
4,14550,张某某,女,汉族,2001.0,白-高二(04),浙江宁波,城镇,共青团员,904,2018-2019-1,NaN,NaN,NaN


In [3]:
consumption.columns=['消费时间','消费金额','学生id','姓名','性别']
max_time=consumption['消费时间'].max()
min_time=consumption['消费时间'].min()
print('最大的消费时间：',max_time)
print('最小的消费时间：',min_time)
consumption.head()

最大的消费时间： 2019/1/9 9:30
最小的消费时间： 2018/10/10 10:14


,消费时间,消费金额,学生id,姓名,性别
0,2018/7/1 6:32,-4.5,14877,张某某,女
1,2018/7/1 11:43,-11.0,14917,高某某,男
2,2018/7/1 14:21,-9.2,14921,胡某某,男
3,2018/7/1 14:33,-6.9,14898,牛某某,男
4,2018/7/1 15:05,-7.0,14917,高某某,男


### 1.1  消费异常值

#### 1.1.1 单次消费最高额

In [5]:
con=consumption
con['学生id']=con['学生id'].astype(str)  # 将学生id转换为字符串，便于画图


# max=con['消费金额'].sort_values(ascending = True)[0:100]  # 只对消费金额这一列进行数值排序，结果只有消费金额和index
max=con.sort_values('消费金额',ascending = True)[0:50] # 对con按照消费金额排序，结果还有其他列
from pyecharts import Bar
bar=Bar('单次消费最高额')
bar.add('消费金额',max['姓名'],np.abs(max['消费金额']),mark_point=['max','min'],is_label_show=False,is_datazoom_show=True)
bar.render(path='D:\Entertainment\Heaven Pool\consumption_information\单次消费最高额.html')
bar

#### 1.1.2  单次消费最低额

In [6]:
# 筛选单次消费金额<1
min=con.ix[con['消费金额']>-1]
least=min.groupby(['学生id'])['消费金额'].agg(['count', 'mean','sum'])
least=least.sort_values('count',ascending = False)[0:50]
least=least.reset_index()
min_index=least['学生id'].values
min_count=least['count'].values
min_mean=np.abs(least['mean'].values)

from pyecharts import Bar,Line,Grid
bar_top=Bar('单次消费额<1时，消费次数最多的学生')
bar_top.add('消费次数',min_index,min_count,mark_point=['max','min'],mark_line=['average'], mark_line_symbolsize=20,legend_top='top',legend_pos='70%',
            is_datazoom_show=True,datazoom_xaxis_index=[0,1])
# 设置 dataZoom 控制索引为 0,1 的 x 轴，即第一个和第二个
bar_bottom=Bar('对应学生的平均消费额', title_top="50%")
bar_bottom.add('平均消费额',min_index,min_mean,mark_line=['average'],legend_top='top',legend_pos='80%', mark_line_symbolsize=20,is_datazoom_show=True)
grid = Grid(width=900, height=500)
grid.add(bar_top, grid_bottom="60%")
grid.add(bar_bottom, grid_top="60%")
grid.render(path='D:\Entertainment\Heaven Pool\consumption_information\最低额.html')
grid

### 1.2 学生消费区间

#### 1.2.1  单次学生消费区间

In [7]:
con1=consumption
con1['消费金额']=np.abs(con['消费金额'])
con1['金额分组']=con1['消费金额']


# 分组依据，注意最小值要减1，最大值要加1，因为pandas的数值分组是左开右闭，或左闭右开
bins=[0,1,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100]
con1['金额分组']=pd.cut(con1['金额分组'].values,
                   bins,
                   right=False)
#使用pandas中的cut进行数值分组，right=False表示左闭右开，省略参数right表示左开右闭


fenzu=con1.groupby(['金额分组'])['消费时间'].count()

from pandas import Series, DataFrame
fenzu.index = Series(['1','5','10','15', '20','25','30','35','40','45','50','55','60','65','70','75','80','85','90','95','100'])

from pyecharts import Scatter
scatter=Scatter('单次消费金额区间分布')
scatter.add('次数',fenzu.index,fenzu.values, is_visualmap=True,visual_type="size",visual_range_size=[4,40],visual_orient="horizontal")
#利用 Visualmap 组件，通过图形点大小映射数值:visual_type="size",visual_range_size=[20,50]
scatter.render(path='D:\Entertainment\Heaven Pool\consumption_information\单次消费区间分布.html')
scatter

#### 1.2.1  学生总消费区间

In [8]:
from pandas import Series, DataFrame
con2=con1.groupby(['学生id'])['消费金额'].sum()
con2=con2.reset_index()
con2['消费金额']=np.ceil(con2['消费金额'])
bins=[0,500,1000,1500,2000,2500,3000,3500,4000,4500,5000,5500,6000,6500]
con2['金额分组']=pd.cut(con2['消费金额'],bins,right=False)
fenzu2=con2.groupby(['金额分组'])['学生id'].count()
fenzu2
fenzu2.index = Series(['500','1000','1500','2000','2500','3000','3500','4000','4500','5000','5500','6000','6500'])
scatter=Scatter('总消费金额区间分布')
scatter.add('次数',fenzu2.index,fenzu2.values,is_visualmap=True,visual_type="size",visual_range_size=[10,40],visual_orient="horizontal")
scatter.render(path='D:\Entertainment\Heaven Pool\consumption_information\总消费金额区间.html')
#利用 Visualmap 组件，通过图形点大小映射数值:
scatter

### 1.3  个人消费总次数和平均消费额

In [9]:
xuesheng=con[['姓名','学生id']].drop_duplicates() # 对学生姓名和学生id去重
# 消费次数最多count的学生
mean=con.groupby(['学生id'])['消费金额'].agg(['count', 'mean','sum'])
max_count=mean.sort_values('count',ascending = False)[0:50]
hebing=pd.merge(max_count,xuesheng,on='学生id')
hebing.head()

,学生id,count,mean,sum,姓名
0,16038,657,7.732725,5080.40,王某某
1,15764,635,6.240551,3962.75,钱某某
2,15679,631,6.948288,4384.37,董某某
3,15743,621,6.681079,4148.95,刘某某
4,16051,600,6.470183,3882.11,吕某某


In [10]:
# 倒映直角坐标系
from pyecharts import Bar,Line,Grid
index=hebing['学生id'].values
mean=np.ceil(np.abs(hebing['mean'].values))

bar_top=Bar('个人消费总次数及平均消费额')
bar_top.add('消费总次数',index,hebing['count'].values,legend_pos="30%",mark_point=['max','min'],xaxis_name='学生id',xaxis_name_pos='start',
           is_datazoom_show=True,datazoom_xaxis_index=[0,1])

line_bottom=Line()
line_bottom.add("平均消费额",index,mean,mark_point=['max','min'],label_color=['#404a59'],is_yaxis_inverse=True,xaxis_pos="top")

grid=Grid(width=900,height=500)
grid.add(bar_top,grid_bottom='60%')
grid.add(line_bottom,grid_top='50%')
grid.render(path=r'D:\Entertainment\Heaven Pool\consumption_information\个人消费总次数及平均消费额1.html')
grid

In [11]:
# 改变轴索引，使其有多x轴或者y轴
from pyecharts import Bar,Line,Overlap,Grid
index1=hebing['学生id'].values
mean1=np.ceil(np.abs(hebing['mean'].values))

bar=Bar('个人消费总次数及平均消费额')
bar.add('消费总次数',index1,hebing['count'].values,xaxis_name='学生id',mark_point=['max','min'],label_color=['#C0C0C0'],xaxis_name_pos='start',is_datazoom_show=True)

line=Line()
line.add("平均消费额",index1,mean1,mark_point=['max','min'],label_color=['#87CEEB'],yaxis_max=14,yaxis_min=0,yaxis_force_interval=2)

overlap = Overlap(width=900, height=400)
overlap.add(bar)# 默认不新增 x y 轴，并且 x y 轴的索引都为 0
# 新增一个 y 轴，此时 y 轴的数量为 2，第二个 y 轴的索引为 1（索引从 0 开始），所以设置 yaxis_index = 1
# 由于使用的是同一个 x 轴，所以 x 轴部分不用做出改变
overlap.add(line, yaxis_index=1, is_add_yaxis=True)
overlap.render(path='D:\Entertainment\Heaven Pool\consumption_information\个人消费总次数及平均消费额2.html')
overlap

### 1.4  贫困生消费情况(将单次消费小于1元的异常值删除)
筛选贫困生标准：消费次数多，但是消费总金额低。定义标准：每个月在食堂超过60顿，一个月消费不足420元的，被列为受资助对象

In [12]:
con3=consumption
con3=con.ix[con['消费金额']>1]
print('原始数据的数据类型：',con3.dtypes)
print("******************************************")
con3['消费时间']=pd.to_datetime(con3['消费时间'])  # 转化时间类型
print('转换时间类型后的数据类型：',con3.dtypes)
print('最小的消费金额：',con3['消费金额'].values.min())

原始数据的数据类型： 消费时间      object
消费金额     float64
学生id      object
姓名        object
性别        object
金额分组    category
dtype: object
******************************************
转换时间类型后的数据类型： 消费时间    datetime64[ns]
消费金额           float64
学生id            object
姓名              object
性别              object
金额分组          category
dtype: object
最小的消费金额： 1.05


In [13]:
# 提取年,月，日
con3['年']=con3['消费时间'].dt.year
con3['月']=con3['消费时间'].dt.month
con3['日']=con3['消费时间'].dt.day

con_month=con3.groupby(['年','月','学生id'])['消费金额'].agg(['sum','count','mean'])
con_month.reset_index(inplace=True)  # inplace将数值替换到原数上

# 将每月总消费额小于400，总消费次数大于50的同学定义为贫困生（每个月在食堂超过50顿，一个月消费不足400元的，被列为受资助对象）
a1=con_month[(con_month['sum']<400) & (con_month['count']>50)]
pinlv=a1.groupby(['学生id'])['月'].count()
pinlv.sort_values(ascending = False,inplace=True)
pinkun=pinlv.iloc[pinlv.values>=1]

# 输出贫困生名单
b=pinkun.reset_index()
#con4=con3[['学生id','姓名']].drop_duplicates()
student_info['学生id']=student_info['学生id'].astype(str)
xuesheng=pd.merge(student_info,b,on='学生id')
xuesheng.rename(columns={'月':'月份数'},inplace = True)
table=xuesheng[['学生id','学生姓名','班级名','月份数']]
table.to_excel(r'D:\Entertainment\Heaven Pool\贫困生名单.xls')

from pyecharts import Line
index=pinkun.index
value=pinkun.values
line=Line('当前学期贫困生','贫困生：每月总消费额>400 & 每月总消费次数>50',subtitle_color=['#FFFFF'])
line.add('月份数',index,value,label_color=['#FFFFF'],is_datazoom_show=True,xaxis_name='学生id',xaxis_name_pos='start')
line.render(path='D:\Entertainment\Heaven Pool\consumption_information\当前学期贫困生消费月份数.html')
line

### 拆分字符串

#拆分数组中字符串
for i in range(con.shape[0]):
    strlist=con.iloc[i,0].split('/')# 用/分割str字符串，并保存到列表
    con.iloc[i,6]=strlist[1]
        

#for i in range(con.shape[0]):
    #con.iloc[i,0].split('/')
str = con.iloc[1,0]
strlist = str.split('/')# 用逗号分割str字符串，并保存到列表
strlist
#for value in strlist:# 循环输出列表值
    #print (value)
con.iloc[1,5]=strlist[0]
con.iloc[1,6]=strlist[1]



### 小插曲1：修改dataframe中某一列的数据类型

In [247]:
# 最后结果只有一列
con1=consumption
con1=str(con1['学生id'].values)
type(con1)

str

In [229]:
# 最后结果是dataframe
con=consumption
con['学生id']=con['学生id'].astype(str)
con.dtypes

消费时间     object
消费金额    float64
学生id     object
姓名       object
性别       object
dtype: object

### 小插曲2：对dataframe中某一列数据整体取整

In [249]:
con=consumption
con['消费金额']=np.abs(con['消费金额'])
con.head()

,消费时间,消费金额,学生id,姓名,性别
0,2018/7/1 6:32,4.5,14877,张某某,女
1,2018/7/1 11:43,11.0,14917,高某某,男
2,2018/7/1 14:21,9.2,14921,胡某某,男
3,2018/7/1 14:33,6.9,14898,牛某某,男
4,2018/7/1 15:05,7.0,14917,高某某,男


### 小插曲3：pandas的分段函数cut():https://blog.csdn.net/weixin_38168620/article/details/81189278

对连续性的数据变量进行分段汇总。


pandas.cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False)

In [271]:
#导入pandas库
import pandas as pd
#导入EXCEL表数值分组数据
df=[34,75,47,98,56,78,90]

#分组依据，注意最小值要减1，最大最要加1.因为pandas的数值分组是左开右闭，或左闭右开
#使用了开区间、闭区间的概念，可百度了解
bins=[-1,60,75,90,101]
 
#分组对应的标签，-1到60对应不及格，60到75对应及格……
labels=['不及格','及格','良好','优秀']
#使用pandas中的cut进行数值分组，right=False表示左闭右开，省略参数right表示左开右闭
df=pd.cut(
        df,
        bins,
        right=False)

### 小插曲4：DataFrame替换  链接：https://blog.csdn.net/longwei92/article/details/83275604

In [297]:
df = pd.DataFrame(
        {
            '名称':['产品1','产品2','产品3','产品4','产品5','产品6','产品7','产品8'],
            '数量':['A','0.7','0.8','0.4','0.7','B','0.76','0.28'],
            '金额':['0','0.48','0.33','C','0.74','0','0','0.22'],
            '合计':['D','0.37','0.28','E','0.57','F','0','0.06'],
        }
        )
df

,名称,数量,金额,合计
0,产品1,A,0,D
1,产品2,0.7,0.48,0.37
2,产品3,0.8,0.33,0.28
3,产品4,0.4,C,E
4,产品5,0.7,0.74,0.57
5,产品6,B,0,F
6,产品7,0.76,0,0
7,产品8,0.28,0.22,0.06


In [298]:
df.replace('A', 0.1)##搜索整个DataFrame, 并将所有符合条件的元素全部替换。操作之后，其实原DataFrame是并没有改变的。改变的只是一个复制品。
df['金额'].replace(0.22, inplace=True) #如果需要改变原数据，需要添加常用参数 inplace=True

,名称,数量,金额,合计
0,产品1,0.1,0,D
1,产品2,0.7,0.48,0.37
2,产品3,0.8,0.33,0.28
3,产品4,0.4,C,E
4,产品5,0.7,0.74,0.57
5,产品6,B,0,F
6,产品7,0.76,0,0
7,产品8,0.28,0.22,0.06


In [304]:
df['合计'].replace({'D':0.11111, 'F':0.22222}, inplace=True)#inplace这个参数在一般情况没多大用处，但是如果只替换部分区域时，inplace参数就有用了
#‘合计’这一列中的0，并没有被替换。
#只有‘金额’这一列的0被替换，而且，替换后的结果不需要我们再和原数据进行合并操作，直接体现在原数据中

,名称,数量,金额,合计
0,产品1,A,0,0.11111
1,产品2,0.7,0.48,0.37
2,产品3,0.8,0.33,0.28
3,产品4,0.4,C,E
4,产品5,0.7,0.74,0.57
5,产品6,B,0,0.22222
6,产品7,0.76,0,0
7,产品8,0.28,0.22,0.06


In [305]:
df['名称'].str.replace('产品', 'product')#只需要替换某个数据的部分内容

0    product1
1    product2
2    product3
3    product4
4    product5
5    product6
6    product7
7    product8
Name: 名称, dtype: object

### 小插曲5：pandas中DataFrame修改index、columns名的方法：https://blog.csdn.net/missyougoon/article/details/83780845

### 小插曲6：时间类型的字符串转化为时间

In [29]:
# 单个字符串转换
import datetime
str_p = '2019-01-30 15:29:08'
dateTime_p = datetime.datetime.strptime(str_p,'%Y-%m-%d %H:%M:%S')
print(dateTime_p)

2019-01-30 15:29:08


In [30]:
# Series转换
time_ser = pd.to_datetime(con3['消费时间']) 

### 小插曲7:python pandas从datetime中提取年份 http://www.voidcn.com/article/p-kjkikhmw-bte.html

### 总结：数据类型是int是无法利用折线图画图的，要将数据类型转换为str